In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import numpy as np

In [3]:
print("Hello World")

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.Hello World



In [4]:
sc.settings.verbosity = 3  
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=True, figsize=(5, 5))

-----
anndata     0.9.1
scanpy      1.9.3
-----
PIL                         9.1.1
anyio                       NA
asciitree                   NA
attr                        21.2.0
babel                       2.9.1
backcall                    0.2.0
brotli                      NA
certifi                     2022.12.07
cffi                        1.15.0
charset_normalizer          2.0.7
cloudpickle                 2.2.0
colorama                    0.4.6
cycler                      0.10.0
cython_runtime              NA
cytoolz                     0.12.1
dask                        2023.5.1
dateutil                    2.8.2
debugpy                     1.5.0
decorator                   5.1.1
defusedxml                  0.7.1
dill                        0.3.6
entrypoints                 0.3
fasteners                   0.18
fastjsonschema              NA
google                      NA
h5py                        3.7.0
idna                        3.3
igraph                      0.10.3
ipykernel 

In [ ]:
# samples_dict = {"Immature": "A0021_043", "0hr": "A0008_041", "1hr": "A0021_044", "4hr": "A0008_045", "4hr_replicate": "A0021_042", "6hr": "A0021_045", "8hr": "A0008_044", "8hr_replicate": "A0021_038", "11hr": "A0008_046", "12hr": "A0021_046"}
samples_dict = {"Immature": "A0021_043", "0hr": "A0008_041", "1hr": "A0021_044", "4hr": "A0021_042", "6hr": "A0021_045", "8hr": "A0008_044", "11hr": "A0008_046", "12hr": "A0021_046"}

In [ ]:
for sample in samples_dict.keys():
        print(sample + ": " + samples_dict[sample])
        adata = sc.read_10x_mtx(path="./../STARsolo_counts/" + sample + "/" + sample + "Solo.out/Gene/raw/")
        print(adata.shape)
        adata.obs_names = [(''.join(x.split("_"))) for x in adata.obs_names]
        barcodes_adata = pd.read_csv("./../whitelists/" + samples_dict[sample] + "_BeadBarcodes.txt", delimiter="\t", names= ["barcode", "x", "y"])
        barcodes_adata[["x", "y"]].to_numpy()
        barcodes_adata.index = barcodes_adata["barcode"]
        print(barcodes_adata.shape)
        shared_barcodes_adata = list(set(adata.obs_names).intersection(set(barcodes_adata["barcode"])))
        adata = adata[shared_barcodes_adata]
        barcodes_adata = barcodes_adata.reindex(adata.obs_names)
        adata.obs = adata.obs.join(barcodes_adata)
        adata.obsm["spatial"] = barcodes_adata[["x", "y"]].to_numpy()
        adata.obs_names_make_unique()
        adata.var_names_make_unique()

        adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
        sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

        sc.settings.set_figure_params(dpi_save= 400, fontsize=6, figsize=(3.0,3.0), facecolor='white', frameon=False, transparent=True, format="pdf")
        print(sample)
        sc.pl.spatial(
                adata,
                color= ["total_counts", "n_genes_by_counts", "pct_counts_mt"],
                spot_size=30, vmin=0, vmax='p99.2')
        sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=True, figsize=(5, 5))
        print(adata.shape)
        adata.write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_raw_counts.h5ad")

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../../pyobjs_github/adata_"+ sample + "_with_image_raw_counts.h5ad")
adata_dict

In [ ]:
for sample in adata_dict.keys():
#         sc.settings.set_figure_params(dpi_save= 400, fontsize=6, figsize=(3.0,3.0), facecolor='white', frameon=False, transparent=True, format="pdf")
#         sc.pl.spatial(
#                 adata_dict[sample],
#                 color= ["total_counts", "n_genes_by_counts"],
#                 spot_size=30, vmin=0, vmax='p99.2', wspace=0.1)
#         sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=True, figsize=(5, 5))
        print(adata_dict[sample].shape)
        sc.pp.filter_cells(adata_dict[sample], min_counts=100)
        print(adata_dict[sample].shape)

In [ ]:
# for sample in adata_dict.keys():
#         print(adata_dict[sample].shape)
#         adata_dict[sample].write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_raw_counts_filtered.h5ad")

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../../pyobjs_github/adata_"+ sample + "_with_image_raw_counts_filtered.h5ad")
adata_dict

In [ ]:
adata_ovary_combined_raw_counts = adata_dict[list(samples_dict.keys())[0]].concatenate([adata_dict[x] for x in list(samples_dict.keys())[1:]], batch_key= "Sample", batch_categories=list(samples_dict.keys()))

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=10, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
grouping_var = "total_counts"
fig, axs = plt.subplots(2,4, figsize = (20,8))
for i, sample in enumerate(samples_dict.keys()):
    subset = adata_ovary_combined_raw_counts[adata_ovary_combined_raw_counts.obs["Sample"] == sample]
    sc.pl.spatial(subset, color = [grouping_var], wspace= 0.0, hspace = 0.0, spot_size = 30, frameon=False, title=sample, show=False, ax=axs[int(i/4), int(i%4)], vmin=0, vmax='p99.2')  
plt.subplots_adjust(wspace=0.1, hspace=0.05)

In [ ]:
# adata_ovary_combined_raw_counts = adata_dict[list(samples_dict.keys())[0]].concatenate([adata_dict[x] for x in list(samples_dict.keys())[1:]], batch_key= "Sample", batch_categories=list(samples_dict.keys()))
# adata_ovary_combined_raw_counts.write_h5ad("./../../pyobjs_github/adata_ovary_combined_raw_counts_filtered.h5ad")
adata_ovary_combined_raw_counts = sc.read_h5ad("./../../pyobjs_github/adata_ovary_combined_raw_counts_filtered.h5ad")
adata_ovary_combined_raw_counts

In [ ]:
for sample in adata_dict.keys():
        sc.pp.normalize_total(adata_dict[sample], inplace=True)
        sc.pp.log1p(adata_dict[sample])
        sc.pp.highly_variable_genes(adata_dict[sample], max_mean=3, min_disp=0.2)
        sc.pl.highly_variable_genes(adata_dict[sample])
        adata_dict[sample].raw = adata_dict[sample]
        adata_dict[sample] = adata_dict[sample][:, adata_dict[sample].var.highly_variable]
        print(adata_dict[sample].shape)
        sc.pp.regress_out(adata_dict[sample], ["total_counts"])
        sc.pp.scale(adata_dict[sample], max_value=10)
        sc.pp.pca(adata_dict[sample], random_state= 0)
        sc.pl.pca_variance_ratio(adata_dict[sample], n_pcs = 50)
        sc.pp.neighbors(adata_dict[sample], n_pcs=20)
        sc.tl.umap(adata_dict[sample])
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.0", resolution = 1.0)
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.2", resolution = 1.2)
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.5", resolution = 1.5)

In [ ]:
adata_ovary_combined_raw_counts = adata_dict[list(samples_dict.keys())[0]].concatenate([adata_dict[x] for x in list(samples_dict.keys())[1:]], batch_key= "Sample", batch_categories=list(samples_dict.keys()))

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=10, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
grouping_var = "leiden_1.5"
fig, axs = plt.subplots(2,5, figsize = (20,8))
for i, sample in enumerate(samples_dict.keys()):
    subset = adata_ovary_combined_raw_counts[adata_ovary_combined_raw_counts.obs["Sample"] == sample]
    sc.pl.spatial(subset, color = [grouping_var], wspace= 0.0, hspace = 0.0, spot_size = 30, frameon=False, title=sample, show=False, ax=axs[int(i/5), int(i%5)], vmin=0, vmax='p99.2')  
plt.subplots_adjust(wspace=0.3, hspace=0.05)

In [ ]:
for sample in adata_dict.keys():
        sc.pl.spatial(adata_dict[sample], color=["leiden_1.0", "leiden_1.2", "leiden_1.5"], spot_size=30, wspace=0.25)

In [ ]:
for sample in adata_dict.keys():
        sc.pl.spatial(adata_dict[sample], color=["leiden_1.0", "leiden_1.2", "leiden_1.5"], spot_size=30, wspace=0.25)

In [ ]:
sc.pl.spatial(adata_dict["Immature"][adata_dict["Immature"].obs["leiden_1.2"].isin(["0", "4"])], color=["leiden_1.2"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["Immature"][~adata_dict["Immature"].obs["leiden_1.2"].isin(["0", "4"])], color=["leiden_1.2"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["Immature"][adata_dict["Immature"].obs["leiden_1.5"].isin(["0", "2", "11"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["Immature"][~adata_dict["Immature"].obs["leiden_1.5"].isin(["0", "2", "11"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save= 300, fontsize=10, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
grouping_var = "Selection"
fig, axs = plt.subplots(1,3, figsize = (20,8))
sc.pl.spatial(adata_dict["0hr"], color=["leiden_1.5"], spot_size=30, ax = axs[0], show = False, title = "All clusters")
sc.pl.spatial(adata_dict["0hr"][adata_dict["0hr"].obs["leiden_1.5"].isin(["0", "1", "2", "6", "11"])], color=["leiden_1.5"], spot_size=30, ax = axs[1], show= False, title = "Smear clusters")
sc.pl.spatial(adata_dict["0hr"][~adata_dict["0hr"].obs["leiden_1.5"].isin(["0", "1", "2", "6", "11"])], color=["leiden_1.5"], spot_size=30, ax = axs[2], show= False, title =  "Filtered data" )
plt.subplots_adjust(wspace=0.2, hspace=0)
plt.show()

In [ ]:
sc.pl.spatial(adata_dict["0hr"][adata_dict["0hr"].obs["leiden_1.5"].isin(["0", "1", "2", "6", "11"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["0hr"][~adata_dict["0hr"].obs["leiden_1.5"].isin(["0", "1", "2", "6", "11"])], color=["leiden_1.5"], spot_size=30, crop_coord=(1300, 5740, 755, 5000), wspace=0.3)

In [ ]:
sc.pl.spatial(adata_dict["1hr"][adata_dict["1hr"].obs["leiden_1.5"].isin(["0", "2", "3"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["1hr"][~adata_dict["1hr"].obs["leiden_1.5"].isin(["0", "3", "2"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["4hr"][adata_dict["4hr"].obs["leiden_1.5"].isin(["4", "2", "6", "9"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["4hr"][~adata_dict["4hr"].obs["leiden_1.5"].isin(["4", "2", "6", "9"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["4hr_replicate"][adata_dict["4hr_replicate"].obs["leiden_1.5"].isin(["0","1", "8", "5"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["4hr_replicate"][~adata_dict["4hr_replicate"].obs["leiden_1.5"].isin(["0","1", "8", "5"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["6hr"][adata_dict["6hr"].obs["leiden_1.5"].isin(["0", "2", "4", "5", "8", "9"])], color=["leiden_1.5"], spot_size=30)


In [ ]:
sc.pl.spatial(adata_dict["6hr"][~adata_dict["6hr"].obs["leiden_1.5"].isin(["0", "2", "4", "5", "8", "9"])], color=["leiden_1.5"], spot_size=30)


In [ ]:
sc.pl.spatial(adata_dict["8hr"][adata_dict["8hr"].obs["leiden_1.5"].isin(["0", "2", "6", "4", "10", "8", "12", "17"])], color=["leiden_1.5"], spot_size=30)


In [ ]:
sc.pl.spatial(adata_dict["8hr"][~adata_dict["8hr"].obs["leiden_1.5"].isin(["0", "2", "6", "4", "10", "8", "12", "17"])], color=["leiden_1.5"], spot_size=30, crop_coord=(1400, 5500, 700, 4800), wspace=0.4)


In [ ]:
sc.pl.spatial(adata_dict["8hr_replicate"][adata_dict["8hr_replicate"].obs["leiden_1.5"].isin(["0", "3", "7"])], color=["leiden_1.5"], spot_size=30)


In [ ]:
sc.pl.spatial(adata_dict["8hr_replicate"][~adata_dict["8hr_replicate"].obs["leiden_1.5"].isin(["0", "3", "7"])], color=["leiden_1.5"], spot_size=30)


In [ ]:
sc.pl.spatial(adata_dict["11hr"][adata_dict["11hr"].obs["leiden_1.5"].isin(["0", "2", "3"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["11hr"][~adata_dict["11hr"].obs["leiden_1.5"].isin(["0", "2", "3"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["12hr"][adata_dict["12hr"].obs["leiden_1.5"].isin(["0", "4", "10"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
sc.pl.spatial(adata_dict["12hr"][~adata_dict["12hr"].obs["leiden_1.5"].isin(["0", "4", "10"])], color=["leiden_1.5"], spot_size=30)

In [ ]:
adata_dict["Immature"] = adata_dict["Immature"][~adata_dict["Immature"].obs["leiden_1.5"].isin(["0", "2", "11"])]
adata_dict["0hr"] = adata_dict["0hr"][~adata_dict["0hr"].obs["leiden_1.5"].isin(["0", "1", "2", "6", "11"])]
adata_dict["1hr"] = adata_dict["1hr"][~adata_dict["1hr"].obs["leiden_1.5"].isin(["0", "3", "2"])]
adata_dict["4hr"] = adata_dict["4hr"][~adata_dict["4hr"].obs["leiden_1.5"].isin(["4", "2", "6", "9"])]
adata_dict["4hr_replicate"] = adata_dict["4hr_replicate"][~adata_dict["4hr_replicate"].obs["leiden_1.5"].isin(["0","1", "8", "5"])]
adata_dict["6hr"] = adata_dict["6hr"][~adata_dict["6hr"].obs["leiden_1.5"].isin(["0", "2", "4", "5", "8", "9"])]
adata_dict["8hr"] = adata_dict["8hr"][~adata_dict["8hr"].obs["leiden_1.5"].isin(["0", "2", "6", "4", "10", "8", "12", "17"])]
adata_dict["8hr_replicate"] = adata_dict["8hr_replicate"][~adata_dict["8hr_replicate"].obs["leiden_1.5"].isin(["0", "3", "7"])]
adata_dict["11hr"] = adata_dict["11hr"][~adata_dict["11hr"].obs["leiden_1.5"].isin(["0", "2", "3"])]
adata_dict["12hr"] = adata_dict["12hr"][~adata_dict["12hr"].obs["leiden_1.5"].isin(["0", "4", "10"])]

In [ ]:
for sample in adata_dict.keys():
        print(adata_dict[sample].shape)
        adata_dict[sample].write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue.h5ad")

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue.h5ad")
        adata_dict[sample].uns['log1p']["base"] = None
adata_dict

In [ ]:
import scipy.spatial as scisp
for sample in samples_dict.keys():
    # Calculate euclidean distances
    adata_dict[sample].obsm["spatial_distances"] = scisp.distance.squareform(scisp.distance.pdist(adata_dict[sample].obsm['spatial']))
    adata_dict[sample].obs["spatial_neighbors_100_true"] = np.sum(adata_dict[sample].obsm["spatial_distances"] < 100, axis=0)
#     sc.pl.spatial(adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,], color=["leiden_1.5", "spatial_neighbors_100_true"], spot_size=30, wspace=0.25, title="")


In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] < 10,], color=["leiden_1.5", "spatial_neighbors_100_true"], spot_size=30, wspace=0.25, title=sample)
    

In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,], color=["leiden_1.5", "spatial_neighbors_100_true"], spot_size=30, wspace=0.25, title=sample)
    

In [ ]:
for sample in samples_dict.keys():
    adata_dict[sample] = adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,]

In [ ]:
for sample in adata_dict.keys():
        print(adata_dict[sample].shape)
        adata_dict[sample].write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered.h5ad")

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered.h5ad")
        adata_dict[sample].uns['log1p']["base"] = None
        adata_dict

In [ ]:
for sample in samples_dict.keys():
        adata_dict[sample] = adata_dict[sample].raw.to_adata()
        sc.pp.highly_variable_genes(adata_dict[sample], max_mean=3, min_disp=0.20)
        sc.pl.highly_variable_genes(adata_dict[sample])
        adata_dict[sample].raw = adata_dict[sample]
        adata_dict[sample] = adata_dict[sample][:, adata_dict[sample].var.highly_variable]
        print(adata_dict[sample].shape)
        sc.pp.regress_out(adata_dict[sample], ["total_counts"])
        sc.pp.scale(adata_dict[sample], max_value=10)
        sc.pp.pca(adata_dict[sample], random_state= 0)
        sc.pl.pca_variance_ratio(adata_dict[sample], n_pcs = 50)
        sc.pp.neighbors(adata_dict[sample], n_pcs=20)
        sc.tl.umap(adata_dict[sample])
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.0", resolution = 1.0)
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.2", resolution = 1.2)
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.5", resolution = 1.5)

In [ ]:
for sample in adata_dict.keys():
        print(adata_dict[sample].shape)
        adata_dict[sample].write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed.h5ad")


In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed.h5ad")
        adata_dict[sample].uns['log1p']["base"] = None
        adata_dict

In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample], color=["leiden_1.5"], spot_size=30, wspace=0.25, title=sample)
    

In [ ]:
adata_dict["Immature"] = adata_dict["Immature"][~adata_dict["Immature"].obs["leiden_1.5"].isin(["9"])]
# adata_dict["0hr"] = adata_dict["0hr"][~adata_dict["0hr"].obs["leiden_1.5"].isin(["0", "1", "2", "6", "11"])]
# adata_dict["1hr"] = adata_dict["1hr"][~adata_dict["1hr"].obs["leiden_1.5"].isin(["0", "3", "2"])]
adata_dict["4hr"] = adata_dict["4hr"][~adata_dict["4hr"].obs["leiden_1.5"].isin(["4"])]
adata_dict["4hr_replicate"] = adata_dict["4hr_replicate"][~adata_dict["4hr_replicate"].obs["leiden_1.5"].isin(["3"])]
# adata_dict["6hr"] = adata_dict["6hr"][~adata_dict["6hr"].obs["leiden_1.5"].isin(["0", "2", "4", "5", "8", "9"])]
# adata_dict["8hr"] = adata_dict["8hr"][~adata_dict["8hr"].obs["leiden_1.5"].isin(["0", "2", "6", "4", "10", "8", "12", "17"])]
adata_dict["8hr_replicate"] = adata_dict["8hr_replicate"][~adata_dict["8hr_replicate"].obs["leiden_1.5"].isin(["5", "9"])]
adata_dict["11hr"] = adata_dict["11hr"][~adata_dict["11hr"].obs["leiden_1.5"].isin(["1", "5"])]
adata_dict["12hr"] = adata_dict["12hr"][~adata_dict["12hr"].obs["leiden_1.5"].isin(["2", "4"])]

In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample], color=["leiden_1.5"], spot_size=30, wspace=0.25, title=sample)
    

In [ ]:
import scipy.spatial as scisp
for sample in samples_dict.keys():
    # Calculate euclidean distances
    adata_dict[sample].obsm["spatial_distances"] = scisp.distance.squareform(scisp.distance.pdist(adata_dict[sample].obsm['spatial']))
    adata_dict[sample].obs["spatial_neighbors_100_true"] = np.sum(adata_dict[sample].obsm["spatial_distances"] < 100, axis=0)
#     sc.pl.spatial(adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,], color=["leiden_1.5", "spatial_neighbors_100_true"], spot_size=30, wspace=0.25, title="")
#     adata_dict[sample] = adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,]

In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] < 10,], color=["leiden_1.5", "spatial_neighbors_100_true"], spot_size=30, wspace=0.25, title=sample)
    

In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,], color=["leiden_1.5", "spatial_neighbors_100_true"], spot_size=30, wspace=0.25, title=sample)
    

In [ ]:
for sample in samples_dict.keys():
    adata_dict[sample] = adata_dict[sample][adata_dict[sample].obs["spatial_neighbors_100_true"] > 10,]

In [ ]:
for sample in adata_dict.keys():
        print(adata_dict[sample].shape)
        adata_dict[sample].write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed_refiltered.h5ad")

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed_refiltered.h5ad")
adata_dict

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../pyobjs/adata_"+ sample + "_with_image_raw_counts_filtered.h5ad")[sc.read_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed_refiltered.h5ad").obs_names]    
adata_dict

In [ ]:
for sample in samples_dict.keys():
        sc.pp.normalize_total(adata_dict[sample], target_sum=1e4, inplace=True)
        sc.pp.log1p(adata_dict[sample])
        sc.pp.highly_variable_genes(adata_dict[sample], max_mean=3)
        sc.pl.highly_variable_genes(adata_dict[sample])
        adata_dict[sample].raw = adata_dict[sample]
        adata_dict[sample] = adata_dict[sample][:, adata_dict[sample].var.highly_variable]
        print(adata_dict[sample].shape)
        sc.pp.regress_out(adata_dict[sample], ["total_counts"])
        sc.pp.scale(adata_dict[sample], max_value=10)
        sc.pp.pca(adata_dict[sample], random_state= 0)
        sc.pl.pca_variance_ratio(adata_dict[sample], n_pcs = 50)
        sc.pp.neighbors(adata_dict[sample], n_pcs=20)
        sc.tl.umap(adata_dict[sample])
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.0", resolution = 1.0)
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.2", resolution = 1.2)
        sc.tl.leiden(adata_dict[sample], key_added="leiden_1.5", resolution = 1.5)

In [ ]:
for sample in samples_dict.keys():
    sc.pl.spatial(adata_dict[sample], color=["leiden_1.0", "leiden_1.2", "leiden_1.5"], spot_size=30, wspace=0.25, title=sample)

In [ ]:
for sample in adata_dict.keys():
        print(adata_dict[sample].shape)
        adata_dict[sample].write_h5ad("./../pyobjs/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed_refiltered_processed.h5ad")

In [ ]:
# samples_dict = {"Immature": "A0021_043", "0hr": "A0008_041", "1hr": "A0021_044", "4hr": "A0008_045", "4hr_replicate": "A0021_042", "6hr": "A0021_045", "8hr": "A0008_044", "8hr_replicate": "A0021_038", "11hr": "A0008_046", "12hr": "A0021_046"}
samples_dict = {"Immature": "A0021_043", "0hr": "A0008_041", "1hr": "A0021_044", "4hr": "A0021_042", "6hr": "A0021_045", "8hr": "A0008_044", "11hr": "A0008_046", "12hr": "A0021_046"}

In [ ]:
adata_dict = dict()
for sample in samples_dict.keys():
        adata_dict[sample] = sc.read_h5ad("./../../pyobjs_github/adata_"+ sample + "_with_image_processed_undertissue_filtered_reprocessed_refiltered_processed.h5ad")
# adata_dict

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=10, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
grouping_var = "total_counts"
fig, axs = plt.subplots(2,4, figsize = (20,8))
for i, sample in enumerate(samples_dict.keys()):
    sc.pl.spatial(adata_dict[sample], color = [grouping_var], wspace= 0.0, hspace = 0.0, spot_size = 30, frameon=False, title=sample, show=False, ax=axs[int(i/4), int(i%4)], vmin=0, vmax='p99.2')  
plt.subplots_adjust(wspace=0.1, hspace=0.05)